In [49]:
import numpy as np
randitnum = 15
def getReward(action):
	return np.random.normal(action, 1)
Q = {k:0 for k in range(1, randitnum+1)}
aNums = {k:0 for k in range(1, randitnum+1)}

In [50]:
import random

def greedy(Q):
	return np.argmax([Q[k] for k in Q]) + 1

def getAction(e):
	action = 0
	if random.random() < e:
		action = random.randint(1, 15)
	else:
		action = greedy(Q)
	return action
def updateQ(action, reward):
	Q[action] = Q[action] + 1/aNums[action] * (reward - Q[action])

In [51]:
rewards = []
actions = []
def run(max_step, e):
	for step in range(1, max_step+1):
		action = getAction(e)
		actions.append(action)
		aNums[action] += 1
		reward = getReward(action)
		rewards.append(reward)
		updateQ(action, reward)

In [52]:
run(2000, 0.4)

In [53]:
aNums

{1: 53,
 2: 68,
 3: 66,
 4: 64,
 5: 50,
 6: 41,
 7: 58,
 8: 63,
 9: 54,
 10: 45,
 11: 41,
 12: 47,
 13: 44,
 14: 56,
 15: 1250}

In [21]:
values = [[0 for i in range(5)] for j in range(5)]
def getdirect(a):
	return [(-1, 0), (1, 0), (0, 1), (0, -1)][a]
def newpos(x, y, a):
	dx, dy = getdirect(a)
	nx = x + dx
	ny = y + dy
	if x == 0 and y == 1:
		return 4, 1, 10
	if x == 0 and y == 3:
		return 2, 3, 5
	if nx < 0 or nx >= 5 or ny < 0 or ny >= 5:
		return x, y, -1
	return nx, ny, 0
while True:
	newvalues = [[0 for i in range(5)] for j in range(5)]
	for x in range(5):
		for y in range(5):
			for a in range(4):
				nx, ny, reward = newpos(x, y, a)
				newvalues[x][y] += 0.25 * (reward + 0.9 * values[nx][ny])
	delta = 0
	for x in range(5):
		for y in range(5):
			delta += abs(newvalues[x][y] - values[x][y])
	if delta < 1e-4:
		print(newvalues)
		break
	values = newvalues

[[3.3090299899087663, 8.78932550661897, 4.427652807616279, 5.322401199441085, 1.492212354487409], [1.5216217226302908, 2.992351499157443, 2.250173575792262, 1.9076053117399359, 0.5474363022367601], [0.050856143296821144, 0.73820423214837, 0.6731468850353048, 0.35821982271759384, -0.4031075461826618], [-0.9735586507977186, -0.43546178774483035, -0.35484864168343366, -0.5855714799606567, -1.183041483445346], [-1.857666897631648, -1.3451976215169594, -1.2292336361173568, -1.4228845392602936, -1.975145450105421]]


In [18]:
import numpy as np


#定义grid问题中常用的变量
# 格子尺寸
WORLD_SIZE = 5
# 状态A的位置(下标从0开始,下同)
A_POS = [0, 1]
# 状态A'的位置
A_PRIME_POS = [4, 1]
# 状态B的位置
B_POS = [0, 3]
# 状态B'的位置
B_PRIME_POS = [2, 3]
# 折扣因子
DISCOUNT = 0.9

# 动作集={上,下,左,右}
ACTIONS = [np.array([-1, 0]),
           np.array([1, 0]),
           np.array([0, 1]),
           np.array([0, -1]),
]
# 策略,每个动作等概率
ACTION_PROB = 0.25

def step(state, action):
  
    # 如果当前位置为状态A,则直接跳到状态A',奖励为+10
    if state == A_POS:
        return A_PRIME_POS, 10
    # 如果当前位置为状态B,则直接跳到状态B',奖励为+5
    if state == B_POS:
        return B_PRIME_POS, 5

    state = np.array(state)
    # 通过坐标运算得到后继状态
    next_state = (state + action).tolist()
    x, y = next_state
    # 根据后继状态的坐标判断是否出界
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        # 出界则待在原地,奖励为-1
        reward = -1.0
        next_state = state
    else:
        # 未出界则奖励为0
        reward = 0
    return next_state, reward

def bellman_equation():
    ''' 求解贝尔曼(期望)方程
    '''
    # 初始值函数
    value = np.zeros((WORLD_SIZE, WORLD_SIZE))
    while True:
        new_value = np.zeros(value.shape)
        # 遍历所有状态
        for i in range(0, WORLD_SIZE):
            for j in range(0, WORLD_SIZE):
                # 遍历所有动作
                for action in ACTIONS:
                    # 执行动作,转移到后继状态,并获得立即奖励
                    (next_i, next_j), reward = step([i, j], action)
                    # 贝尔曼期望方程
                    new_value[i, j] += ACTION_PROB * \
                    (reward + DISCOUNT * value[next_i, next_j])
        # 迭代终止条件: 误差小于1e-4
        if np.sum(np.abs(value - new_value)) < 1e-4:
            print(new_value)
            break
        value = new_value

bellman_equation()


[[ 3.30902999  8.78932551  4.42765281  5.3224012   1.49221235]
 [ 1.52162172  2.9923515   2.25017358  1.90760531  0.5474363 ]
 [ 0.05085614  0.73820423  0.67314689  0.35821982 -0.40310755]
 [-0.97355865 -0.43546179 -0.35484864 -0.58557148 -1.18304148]
 [-1.8576669  -1.34519762 -1.22923364 -1.42288454 -1.97514545]]
